**IMPORTS**

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('./datasets/Anti-Shipping_Activity_Messages.csv')
#https://livingatlas-dcdev.opendata.arcgis.com/datasets/esri2::anti-shipping-activity-messages/explore?location=12.331257%2C2.216667%2C2.59&showTable=true

In [3]:
df.head()

,X,Y,OBJECTID,reference,dateofocc,subreg,hostility_d,victim_d,description,hostilitytype_l,victim_l,navarea
0,-75.133333,21.933333,1,1990-9,1990/06/03 00:00:00+00,26,CUBAN GUNBOAT,BELESBAT QUEEN,A CUBAN GUNBOAT COMMANDEERED LUXURY YACHT BELE...,2.0,11.0,IV
1,108.000000,3.000000,2,1990-10,1990/03/20 00:00:00+00,71,PIRATES,RO/RO SEA DRAGON,20 MARCH 1990. BORNEO. ...,1.0,3.0,XI
2,90.000000,-1.000000,3,1990-11,1990/03/20 00:00:00+00,61,PIRATES,RO/RO SUNRISE,20 MARCH 1990. SINGAPORE. ...,1.0,3.0,VIII
3,42.000000,14.000000,4,1989-16,1989/01/01 00:00:00+00,62,PEOPLES DEMOCRATIC REPUBLIC OF YEMEN,U.S. MARINERS,"RED SEA, YEMEN ...",2.0,13.0,IX
4,80.300000,13.100000,5,1989-17,1989/09/23 00:00:00+00,63,PIRATES,LASH STONEWALL JACKSON,Indian ocean ...,1.0,11.0,VIII


In [4]:
df.shape

(8185, 12)

**Data Cleaning**

In [5]:
df.isnull().sum()

X                    0
Y                    0
OBJECTID             0
reference            0
dateofocc            0
subreg               0
hostility_d        208
victim_d           175
description          4
hostilitytype_l     25
victim_l            27
navarea              3
dtype: int64

In [4]:
# Dropping all rows with missing data as the amount as the percentage of nulls is so low. We can decide to do
# otherwise later.
df.dropna(inplace = True)
df.reset_index(inplace = True)

In [5]:
# Reference is the reference ID for the incident. All values unique. The only useable info in this column is the
# year, which can also be found in the date.
df.reference.value_counts().count()
df.drop(columns = ['reference'], inplace = True)

**Feature engineering: 'dateofocc' (Date of Occurence)**

In [6]:
df.dateofocc.value_counts().count()

4786

In [7]:
#Breaking into month, year, day and time.
df['year'] = [x[0:4] for x in df.dateofocc]
df['month'] = [x[5:7] for x in df.dateofocc]
df['day'] = [x[8:10] for x in df.dateofocc]
df['time'] = [x[11:] for x in df.dateofocc]

In [8]:
df[df['time'] == '00:00:00+00'].count()
#most rows are missing 'time'

index              7792
X                  7792
Y                  7792
OBJECTID           7792
dateofocc          7792
subreg             7792
hostility_d        7792
victim_d           7792
description        7792
hostilitytype_l    7792
victim_l           7792
navarea            7792
year               7792
month              7792
day                7792
time               7792
dtype: int64

In [9]:
#Dropping time column
df.drop(columns = 'time', inplace = True)

In [10]:
#Dropping original date of occurence column
df.drop(columns = 'dateofocc', inplace = True)

In [11]:
df['OBJECTID'].value_counts().count()

# OBJECTID has all unique values. Can be dropped

7889

In [12]:
df.drop(columns = 'OBJECTID', axis = 1, inplace = True)

In [13]:
#lower case all 'object' columns to reduce redundant values:
for i in df.columns:
    if df[i].dtypes == 'O':
        df[i] = df[i].str.lower()

**Feature engineering: 'hostilitytype_l' (Hostility Type)**

In [14]:
df.hostilitytype_l.value_counts()

1.0    6971
3.0     537
4.0     163
6.0     124
2.0      47
5.0      40
9.0       6
7.0       1
Name: hostilitytype_l, dtype: int64

In [15]:
host_lst = ['pirate_assaults', 
            'navel_engagement', 
            'suspicious_approach', 
            'kidnapping', 
            'unknown', 
            'other', 
            'hijacking', 
            'no entries', 
            'attempted_boarding']

df.hostilitytype_l = [host_lst[int(x) - 1] for x in df.hostilitytype_l]

In [16]:
df.hostilitytype_l.value_counts()

pirate_assaults        6971
suspicious_approach     537
kidnapping              163
other                   124
navel_engagement         47
unknown                  40
attempted_boarding        6
hijacking                 1
Name: hostilitytype_l, dtype: int64

**Feature engineering: 'victim_l' (Victim type)**

In [17]:
df.victim_l.value_counts()

5.0     2960
3.0     1613
9.0     1328
11.0     926
4.0      343
8.0      272
10.0     244
13.0      42
12.0      39
7.0       37
2.0       35
6.0       29
1.0       21
Name: victim_l, dtype: int64

In [18]:
victim_lst = ['anchored_vessel', 
            'barge', 
            'cargo_ship', 
            'fishing_vessel', 
            'merchant_vessel', 
            'offshore_vessel', 
            'passenger_ship', 
            'sailing_vessel', 
            'tanker',
            'tugboat',
            'vessel',
            'unknown',
            'other']

df.victim_l = [victim_lst[int(x) - 1] for x in df.victim_l]

In [19]:
df.victim_l.value_counts()

merchant_vessel    2960
cargo_ship         1613
tanker             1328
vessel              926
fishing_vessel      343
sailing_vessel      272
tugboat             244
other                42
unknown              39
passenger_ship       37
barge                35
offshore_vessel      29
anchored_vessel      21
Name: victim_l, dtype: int64

**Dummy Categorical Columns:**

In [22]:
#Dummy all categorical columns
to_dummy = ['subreg', 'hostility_d', 'victim_d', 'hostilitytype_l', 'navarea']

for i in to_dummy:
    df = pd.concat([df, pd.get_dummies(df[i], prefix = i)], axis=1)
df.drop(columns = to_dummy, axis = 1)

,index,X,Y,description,victim_l,year,month,day,subreg_11,subreg_12,...,navarea_vi,navarea_vii,navarea_viii,navarea_x,navarea_xi,navarea_xii,navarea_xiii,navarea_xiv,navarea_xv,navarea_xvi
0,0,-75.133333,21.933333,a cuban gunboat commandeered luxury yacht bele...,vessel,1990,06,03,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,108.000000,3.000000,20 march 1990. borneo. ...,cargo_ship,1990,03,20,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,90.000000,-1.000000,20 march 1990. singapore. ...,cargo_ship,1990,03,20,0,0,...,0,0,1,0,0,0,0,0,0,0
3,3,42.000000,14.000000,"red sea, yemen ...",other,1989,01,01,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,80.300000,13.100000,indian ocean ...,vessel,1989,09,23,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,8180,120.843200,14.531200,03.09.2020: 1935 utc: posn: 14-31.87n - 120-50...,cargo_ship,2020,09,03,0,0,...,0,0,0,0,1,0,0,0,0,0
7885,8181,13.283300,-8.750000,"angola: on 8 september at 2254lt, robbers boar...",cargo_ship,2020,09,08,0,0,...,0,1,0,0,0,0,0,0,0,0
7886,8182,117.592300,0.252700,"indonesia: on 25 june, robbers boarded an anch...",cargo_ship,2020,06,25,0,0,...,0,0,0,0,1,0,0,0,0,0
7887,8183,120.866700,14.583300,"philippines: on 5 september at 0220lt, a robbe...",cargo_ship,2020,09,05,0,0,...,0,0,0,0,1,0,0,0,0,0


**Export dataframe to csv**

In [23]:
df.head()

,index,X,Y,subreg,hostility_d,victim_d,description,hostilitytype_l,victim_l,navarea,...,navarea_vi,navarea_vii,navarea_viii,navarea_x,navarea_xi,navarea_xii,navarea_xiii,navarea_xiv,navarea_xv,navarea_xvi
0,0,-75.133333,21.933333,26,cuban gunboat,belesbat queen,a cuban gunboat commandeered luxury yacht bele...,navel_engagement,vessel,iv,...,0,0,0,0,0,0,0,0,0,0
1,1,108.000000,3.000000,71,pirates,ro/ro sea dragon,20 march 1990. borneo. ...,pirate_assaults,cargo_ship,xi,...,0,0,0,0,1,0,0,0,0,0
2,2,90.000000,-1.000000,61,pirates,ro/ro sunrise,20 march 1990. singapore. ...,pirate_assaults,cargo_ship,viii,...,0,0,1,0,0,0,0,0,0,0
3,3,42.000000,14.000000,62,peoples democratic republic of yemen,u.s. mariners,"red sea, yemen ...",navel_engagement,other,ix,...,0,0,0,0,0,0,0,0,0,0
4,4,80.300000,13.100000,63,pirates,lash stonewall jackson,indian ocean ...,pirate_assaults,vessel,viii,...,0,0,1,0,0,0,0,0,0,0


In [24]:
df.shape

(7889, 1567)

In [20]:
df.to_csv('./datasets/cleaned_pirate_activity_eda.csv')

In [26]:
df.head()

,index,X,Y,subreg,hostility_d,victim_d,description,hostilitytype_l,victim_l,navarea,...,navarea_vi,navarea_vii,navarea_viii,navarea_x,navarea_xi,navarea_xii,navarea_xiii,navarea_xiv,navarea_xv,navarea_xvi
0,0,-75.133333,21.933333,26,cuban gunboat,belesbat queen,a cuban gunboat commandeered luxury yacht bele...,navel_engagement,vessel,iv,...,0,0,0,0,0,0,0,0,0,0
1,1,108.000000,3.000000,71,pirates,ro/ro sea dragon,20 march 1990. borneo. ...,pirate_assaults,cargo_ship,xi,...,0,0,0,0,1,0,0,0,0,0
2,2,90.000000,-1.000000,61,pirates,ro/ro sunrise,20 march 1990. singapore. ...,pirate_assaults,cargo_ship,viii,...,0,0,1,0,0,0,0,0,0,0
3,3,42.000000,14.000000,62,peoples democratic republic of yemen,u.s. mariners,"red sea, yemen ...",navel_engagement,other,ix,...,0,0,0,0,0,0,0,0,0,0
4,4,80.300000,13.100000,63,pirates,lash stonewall jackson,indian ocean ...,pirate_assaults,vessel,viii,...,0,0,1,0,0,0,0,0,0,0
